In [87]:
import pandas as pd
import re
import underthesea
import os

In [88]:
data_path = "/content/drive/MyDrive/Dataset_For_Work"
test_data_1 = "/content/drive/MyDrive/Dataset_For_Work/test_raw_ANS.txt"
test_data_2 = "/content/drive/MyDrive/Dataset_For_Work/test_tokenized_ANS.txt"

In [89]:
def load_data(file_path, label):
    """
    Đọc dữ liệu từ file và gắn nhãn tương ứng.
    Args:
        file_path (str): Đường dẫn đến tệp văn bản.
        label (str): Nhãn cảm xúc (positive, neutral, negative).
    Returns:
        data (list): Danh sách chứa các câu và nhãn.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            sentence = line.strip()  # Xóa khoảng trắng ở đầu và cuối câu
            if sentence:  # Chỉ thêm những câu không rỗng
                data.append((sentence, label))
    return data

In [90]:
def load_test(file_path):
    """
    Đọc dữ liệu từ file và gắn nhãn cảm xúc từ chuỗi đã cho.
    Args:
        file_path (str): Đường dẫn đến tệp văn bản.
    Returns:
        data (list): Danh sách chứa các câu và nhãn tương ứng.
    """
    import re

    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()  # Đọc toàn bộ nội dung file
        split_data = re.split(r'(POS|NEG|NEU)', text.strip())  # Tách dữ liệu theo nhãn
        for i in range(0, len(split_data) - 1, 2):  # Lặp qua từng cặp câu và nhãn
            sentence = split_data[i].strip()  # Xóa khoảng trắng ở đầu và cuối câu
            label = split_data[i + 1]  # Lấy nhãn tương ứng
            if sentence:  # Chỉ thêm những câu không rỗng
                data.append((sentence, label))
    return data

In [91]:
def clean_text(text):
    """
    Làm sạch văn bản bằng cách loại bỏ ký tự đặc biệt, số và khoảng trắng thừa.
    Args:
        text (str): Câu văn bản cần làm sạch.
    Returns:
        str: Văn bản sau khi làm sạch.
    """
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
    return text

In [92]:
from underthesea import word_tokenize
import pandas as pd

# Load stopwords from file
stop_words_path = "/content/drive/MyDrive/Dataset_For_Work/vietnamese-stopwords.txt"

with open(stop_words_path, 'r', encoding='utf-8') as f:
    stop_words = set(f.read().splitlines())

# Function to remove stopwords
def remove_stopwords(sentence):
    # Tokenize and filter stopwords
    word_tokens = word_tokenize(sentence)
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Function to map sentiment labels
def replace_labels(label):  # Changed here: removed 'df' and kept 'label_column'
    sentiment_map = {'POS': 1, 'NEG': -1, 'NEU': 0}
    return sentiment_map.get(label, label)

In [93]:
# Đọc dữ liệu từ các tệp và gắn nhãn
positive_data = load_data(os.path.join(data_path, "SA-training_positive.txt"), "POS")
neutral_data = load_data(os.path.join(data_path, "SA-training_neutral.txt"), "NEU")
negative_data = load_data(os.path.join(data_path, "SA-training_negative.txt"), "NEG")

# Kết hợp tất cả dữ liệu
all_data = positive_data + neutral_data + negative_data

# Tiền xử lý văn bản
cleaned_data = [(clean_text(sentence), label) for sentence, label in all_data]

# Chuyển dữ liệu sang DataFrame
df = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])

# Hiển thị một vài dòng đầu tiên
print(df.head())

                                            Sentence Label
0  đang xài mx dùng bình thường ngon pin trâu mỗi...   POS
1  qủa pin ngon sạc lại được bền riêng em dùng pi...   POS
2  cũng đang xài con logitech bluetooth tầm thấp ...   POS
3  logitech pin trâu thôi rôi mua con b cui ma cu...   POS
4  em có con chuột không dây k cũng đầy đủ nút bấ...   POS


In [94]:
test = load_test(test_data_1)
cleaned_data = [(clean_text(sentence), label) for sentence, label in test]
tf1 = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])
print(tf1.head())

                                            Sentence Label
0  không nên mua chuột cua logitech vì dùng nó rấ...   POS
1  nói thiệt là mình thì thì chuột nào mình cũng ...   NEG
2                  xai chuot so nhat bi double click   NEU
3  cơ bản là thiết kế ôm chuột chưa đã như hiện g...   POS
4  đang dùng mx cũng ngon nhưng chưa đầy năm mà đ...   NEG


In [95]:
test = load_test(test_data_2)
cleaned_data = [(clean_text(sentence), label) for sentence, label in test]
tf2 = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])
print(tf2.head())

                                            Sentence Label
0  không nên mua chuột cua logitech vì dùng nó rấ...   POS
1  nói thiệt là mình thì thì chuột nào mình cũng ...   NEG
2                  xai chuot so nhat bi double_click   NEU
3  cơ_bản là thiết_kế ôm chuột chưa đã như hiện_g...   POS
4  đang dùng mx_ cũng ngon nhưng chưa đầy_năm mà ...   NEG


In [96]:
df['Sentence'] = df['Sentence'].apply(remove_stopwords)
df['Label'] = df['Label'].apply(replace_labels)

# Display updated DataFrame
print("Dữ liệu sau khi xử lý:")
print(df.head())

Dữ liệu sau khi xử lý:
                                            Sentence  Label
0  xài mx bình thường ngon pin trâu tội thằng chỗ...      1
1               pin ngon sạc bền pin aa aaa thay thế      1
2      xài logitech bluetooth tầm m xài đc ngon chán      1
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...      1
4                chuột dây k đầy đủ nút bấm pin thay      1


In [97]:
tf1['Sentence'] = tf1['Sentence'].apply(remove_stopwords)
tf1['Label'] = tf1['Label'].apply(replace_labels)

# Display updated DataFrame
print("Dữ liệu sau khi xử lý:")
print(tf1.head())

Dữ liệu sau khi xử lý:
                                            Sentence  Label
0  mua chuột cua logitech đổi thằng xúi mua m cơ ...      1
1  thiệt chuột tuốt trừ hãng razer sở hữu da blac...     -1
2                     xai chuot nhat bi double click      0
3  cơ bản thiết kế ôm chuột hiện giờ chuột hàng n...      1
4              mx ngon tháo thay nút bấm may lột máy     -1


In [98]:
tf2['Sentence'] = tf2['Sentence'].apply(remove_stopwords)
tf2['Label'] = tf2['Label'].apply(replace_labels)

# Display updated DataFrame
print("Dữ liệu sau khi xử lý:")
print(tf2.head())

Dữ liệu sau khi xử lý:
                                            Sentence  Label
0  mua chuột cua logitech đổi thằng xúi mua m cơ_...      1
1  thiệt chuột tuốt trừ hãng razer sở_hữu da blac...     -1
2                     xai chuot nhat bi double_click      0
3  cơ_bản thiết_kế ôm chuột hiện_giờ chuột hàng n...      1
4     mx_ ngon đầy_năm tháo thay nút bấm may lột máy     -1


In [99]:
from gensim.models import Word2Vec
from underthesea import word_tokenize

# Chuẩn bị dữ liệu cho Word2Vec
# Mỗi câu cần được token hóa thành danh sách từ
tokenized_sentences = df['Sentence'].apply(word_tokenize).tolist()

# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(
    sentences=tokenized_sentences,  # Câu đã được token hóa
    vector_size=100,               # Kích thước vector
    window=5,                      # Kích thước cửa sổ ngữ cảnh
    min_count=1,                   # Bỏ qua từ xuất hiện ít hơn 1 lần
    sg=1,                          # Sử dụng Skip-Gram (0 cho CBOW)
    epochs=10                      # Số lần huấn luyện
)

In [100]:
# Lấy vector của từ "khách_sạn"
word_vector = word2vec_model.wv['pin']
print("Vector của từ 'pin':", word_vector)

Vector của từ 'pin': [ 1.99249506e-01 -4.36593294e-02 -2.01470807e-01  1.06672794e-01
  2.25676030e-01 -4.73399967e-01  4.41484362e-01  4.43093687e-01
 -1.73236012e-01 -7.59447813e-02  1.57781884e-01 -4.06754613e-01
 -2.79064655e-01  4.87020910e-02 -2.33623505e-01 -2.21857816e-01
  4.09822226e-01 -6.14179969e-02 -2.50109434e-01 -5.67974031e-01
 -1.65379883e-04  1.60138056e-01  2.62542725e-01 -9.03877318e-02
 -9.54064578e-02  1.30866066e-01  3.22116278e-02 -2.74598032e-01
 -1.01372883e-01 -1.28232017e-01  4.15105700e-01  3.15618545e-01
  3.47024687e-02 -2.09962666e-01 -3.03575754e-01  7.37473890e-02
  1.36246011e-01 -3.65353376e-01  1.77255854e-01 -7.60315835e-01
  5.00463367e-01 -4.26257640e-01  3.33881110e-01 -2.06038311e-01
  4.96722400e-01  5.84370375e-01 -3.74880552e-01 -2.38962412e-01
 -5.65280579e-02  7.44526908e-02 -1.38454348e-01 -3.19362551e-01
 -2.01504722e-01  1.49517730e-01 -3.17633480e-01 -1.82407070e-02
  2.16003031e-01 -1.26213506e-01 -3.64194661e-01  6.70154244e-02
  3.

In [101]:
import numpy as np

def sentence_to_vector(sentence, model):
    # Tokenize câu
    tokens = word_tokenize(sentence)
    # Lấy vector của từng từ, nếu từ không tồn tại trong từ điển thì bỏ qua
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    # Tính trung bình các vector từ
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Vector rỗng nếu không có từ hợp lệ

In [102]:
# Biểu diễn toàn bộ tập dữ liệu
df['Sentence_Vector'] = df['Sentence'].apply(lambda x: sentence_to_vector(x, word2vec_model))

# Hiển thị một vài dòng đầu tiên
print(df.head())

                                            Sentence  Label  \
0  xài mx bình thường ngon pin trâu tội thằng chỗ...      1   
1               pin ngon sạc bền pin aa aaa thay thế      1   
2      xài logitech bluetooth tầm m xài đc ngon chán      1   
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...      1   
4                chuột dây k đầy đủ nút bấm pin thay      1   

                                     Sentence_Vector  
0  [0.052804887, 0.06910979, -0.14952543, 0.02957...  
1  [0.06292388, 0.028278971, -0.10335348, 0.01718...  
2  [0.12708806, 0.14319688, -0.18849689, 0.043886...  
3  [0.06200254, 0.08732359, -0.099150985, 0.03169...  
4  [0.018859807, 0.08623658, -0.09054096, 0.06014...  


In [103]:
# Biểu diễn toàn bộ tập dữ liệu
tf1['Sentence_Vector'] = tf1['Sentence'].apply(lambda x: sentence_to_vector(x, word2vec_model))

# Hiển thị một vài dòng đầu tiên
print(tf1.head())

                                            Sentence  Label  \
0  mua chuột cua logitech đổi thằng xúi mua m cơ ...      1   
1  thiệt chuột tuốt trừ hãng razer sở hữu da blac...     -1   
2                     xai chuot nhat bi double click      0   
3  cơ bản thiết kế ôm chuột hiện giờ chuột hàng n...      1   
4              mx ngon tháo thay nút bấm may lột máy     -1   

                                     Sentence_Vector  
0  [0.034310948, 0.12954247, -0.10234582, 0.06872...  
1  [0.020708924, 0.104175724, -0.10100166, 0.0522...  
2  [0.05503485, 0.1231741, -0.03557769, 0.0169322...  
3  [0.019779623, 0.12602726, -0.2025885, 0.018201...  
4  [0.01128612, 0.10690278, -0.061254647, 0.00831...  


In [104]:
# Biểu diễn toàn bộ tập dữ liệu
tf2['Sentence_Vector'] = tf2['Sentence'].apply(lambda x: sentence_to_vector(x, word2vec_model))

# Hiển thị một vài dòng đầu tiên
print(tf2.head())

                                            Sentence  Label  \
0  mua chuột cua logitech đổi thằng xúi mua m cơ_...      1   
1  thiệt chuột tuốt trừ hãng razer sở_hữu da blac...     -1   
2                     xai chuot nhat bi double_click      0   
3  cơ_bản thiết_kế ôm chuột hiện_giờ chuột hàng n...      1   
4     mx_ ngon đầy_năm tháo thay nút bấm may lột máy     -1   

                                     Sentence_Vector  
0  [0.038315285, 0.15769373, -0.10229123, 0.09989...  
1  [0.019190429, 0.10466096, -0.10188547, 0.04904...  
2  [0.06594109, 0.13041188, -0.0096508395, 0.0139...  
3  [0.009836002, 0.11983356, -0.16655947, 0.04892...  
4  [0.011864754, 0.10820059, -0.074378096, 0.0155...  


In [105]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

# Step 1: Train Word2Vec model
def train_word2vec(reviews, vector_size=100, window=5, min_count=1):
    tokenized_reviews = [review.split() for review in reviews]
    w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=vector_size, window=window, min_count=min_count, sg=0)
    return w2v_model

# Step 2: Vectorize reviews
def get_sentence_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Train Word2Vec model on sentences
w2v_model = train_word2vec(df['Sentence'], vector_size=100)

# Prepare training data
X_train = [get_sentence_vector(sentence, w2v_model) for sentence in df['Sentence']]
y_train = df['Label']

# Fit Logistic Regression model
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

# Prepare test data
X_test_1 = [get_sentence_vector(sentence, w2v_model) for sentence in tf1['Sentence']]
X_test_2 = [get_sentence_vector(sentence, w2v_model) for sentence in tf2['Sentence']]
y_test_1 = tf1['Label']
y_test_2 = tf2['Label']

In [106]:
# Predict labels for test data
y_pred_1 = classifier.predict(X_test_1)

# Calculate accuracy
accuracy = accuracy_score(y_test_1, y_pred_1)

# Display accuracy
print(f"Accuracy: {accuracy:.2f}")

# Create a table to display results
results_df = pd.DataFrame({
    'Sentence': tf1['Sentence'],
    'True Label': y_test_1,
    'Predicted Label': y_pred_1
})

# Map numeric labels to text (Optional)
label_mapping = {1: 'Positive', -1: 'Negative', 0: 'Neutral'}
results_df['True Label'] = results_df['True Label'].map(label_mapping)
results_df['Predicted Label'] = results_df['Predicted Label'].map(label_mapping)

# Display the table
print("\nPredictions Table:")
print(results_df)

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test_1, y_pred_1, target_names=["Negative", "Neutral", "Positive"]))

Accuracy: 0.48

Predictions Table:
                                               Sentence True Label  \
0     mua chuột cua logitech đổi thằng xúi mua m cơ ...   Positive   
1     thiệt chuột tuốt trừ hãng razer sở hữu da blac...   Negative   
2                        xai chuot nhat bi double click    Neutral   
3     cơ bản thiết kế ôm chuột hiện giờ chuột hàng n...   Positive   
4                 mx ngon tháo thay nút bấm may lột máy   Negative   
...                                                 ...        ...   
1045                                    xấu khủng khiếp   Negative   
1046     mẹ đài loan mua iphone plus i đổi máy chờ quen    Neutral   
1047         tùng minh nguyễn điện thoại vk may cảm ứng    Neutral   
1048  mua g bộ nhớ g ko thẻ yếu chụp ảnh game cân nh...   Negative   
1049  sản phẩm gionee tốt minh dung s camera đêm ko ...   Positive   

     Predicted Label  
0            Neutral  
1           Negative  
2           Positive  
3           Negative  
4        

In [107]:
# Predict labels for test data
y_pred_2 = classifier.predict(X_test_2)

# Calculate accuracy
accuracy = accuracy_score(y_test_2, y_pred_2)

# Display accuracy
print(f"Accuracy: {accuracy:.2f}")

# Create a table to display results
results_df = pd.DataFrame({
    'Sentence': tf2['Sentence'],
    'True Label': y_test_2,
    'Predicted Label': y_pred_2
})

# Map numeric labels to text (Optional)
label_mapping = {1: 'Positive', -1: 'Negative', 0: 'Neutral'}
results_df['True Label'] = results_df['True Label'].map(label_mapping)
results_df['Predicted Label'] = results_df['Predicted Label'].map(label_mapping)

# Display the table
print("\nPredictions Table:")
print(results_df)

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test_2, y_pred_2, target_names=["Negative", "Neutral", "Positive"]))

Accuracy: 0.45

Predictions Table:
                                               Sentence True Label  \
0     mua chuột cua logitech đổi thằng xúi mua m cơ_...   Positive   
1     thiệt chuột tuốt trừ hãng razer sở_hữu da blac...   Negative   
2                        xai chuot nhat bi double_click    Neutral   
3     cơ_bản thiết_kế ôm chuột hiện_giờ chuột hàng n...   Positive   
4        mx_ ngon đầy_năm tháo thay nút bấm may lột máy   Negative   
...                                                 ...        ...   
1045                                    xấu khủng_khiếp   Negative   
1046  mẹ đài_loan mua iphone__plus i như_vậy đổi máy...    Neutral   
1047  tùng_minh_nguyễn điện_thoại vk thế_nào may cảm...    Neutral   
1048  mua g bộ_nhớ_trong g ko thẻ_nhớ điểm_yếu chụp ...   Negative   
1049  sản_phẩm gionee tốt minh dung s_ camera đêm ko...   Positive   

     Predicted Label  
0            Neutral  
1           Negative  
2           Negative  
3            Neutral  
4        